<a href="https://colab.research.google.com/github/xHexlabx/SuperAI_SS4_Recap/blob/main/SuperAI_SS4_Level_1/Hack_1_Image_Search/Image_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuperAI Season 4 - Level 1 Hackathon - Image Search

# Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Kaggle

In [2]:
!pip install -q kaggle

In [3]:
#kaggle setting for competition
from google.colab import userdata

username = userdata.get('KAGGLE_USER')
key = userdata.get('KAGGLE_KEY')
# Echo the credentials into the kaggle.json file
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

# Explore Datasets

In [4]:
import cv2
import numpy as np

In [5]:
def refine_image (image) :

    width = 336
    height = 336

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image , (width , height))
    image = np.array(image)[: , : , 0 : 3]

    return image

In [6]:
refined_image_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/queries/refined_queries'

for i in range(22) :

    image = cv2.imread(f'/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/queries/queries/{i}.jpg')

    resized_image = refine_image(image)

    with open(f'{refined_image_path}/{i}.npy' , 'wb') as file :

        np.save(file , resized_image)

# CLIP VisionModel For Encode Image

In [7]:
!nvidia-smi

Wed May 22 12:36:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [8]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPModel
import torch

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14')
processor = AutoProcessor.from_pretrained('openai/clip-vit-large-patch14')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [10]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [11]:
inputs = processor(images = image, return_tensors = "pt")
image_features = model.get_image_features(**inputs)

In [12]:
image_features.shape

torch.Size([1, 768])

# Solution-CLIP Zeroshot For Image Similarity Search

## Cosine Similarity

In [13]:
import numpy as np
from torch import nn

def cosine_similarity (A , B) :

    A = np.array(A)
    B = np.array(B)

    norm_A = np.linalg.norm(A)

    similarity_matrix = (A @ B.T).astype('float') / norm_A

    for i in range(B.shape[0]) :

        norm_B = np.linalg.norm(B[i])

        similarity_matrix[0][i] /= norm_B

    similarity_matrix = similarity_matrix.reshape(-1)

    # softmax
    # similarity_matrix = np.exp(similarity_matrix)/sum(np.exp(similarity_matrix))

    target_class = similarity_matrix.argmax(axis = 0)
    maximum_similarity = similarity_matrix[target_class]

    return maximum_similarity , target_class

In [14]:
print(cosine_similarity(np.array([[1 , 2  ,3]]) , np.array([[1 , 2 , 3] , [3 , 4 , 5]])))

(1.0, 0)


## Define Model CLIP

In [15]:
from PIL import Image
import requests
from transformers import AutoProcessor , CLIPModel

## openai/clip-vit-large-patch14

In [16]:
import torch

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14').to(device)
processor = AutoProcessor.from_pretrained('openai/clip-vit-large-patch14')

## Read queries folder image and store to numpy array

In [18]:
import cv2
import os
import numpy as np

In [19]:
refined_queries_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/queries/refined_queries'
images = []

for file_name in os.listdir(refined_queries_path) :

    with open(f'{refined_queries_path}/{file_name}' , 'rb') as file :

        image = np.load(file)

    images.append(image)

inputs = processor(images = images , return_tensors = 'pt').to(device)
image_features = model.get_image_features(** inputs)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [20]:
image_features = image_features.cpu().detach().numpy()

In [21]:
with open('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/database/queries_matrix.npy' , 'wb') as file :

    np.save(file , image_features)


## Read test folder images to inference

In [22]:
from tqdm import tqdm
import pandas as pd

In [23]:
with open('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/database/queries_matrix.npy' , 'rb') as file :

    queries_matrix = np.load(file)

In [24]:
queries_matrix.shape

(22, 768)

In [25]:
submissions = {
    'img_file' : [] ,
    'class' : []
}

In [26]:
threshold = 0

In [27]:
test_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/test/images'

for file_name in tqdm(os.listdir(test_path)) :

    image = cv2.imread(f'{test_path}/{file_name}')
    image = refine_image(image)

    inputs = processor(images = image , return_tensors = 'pt').to(device)
    image_features = model.get_image_features(** inputs)

    maximum_similarity , target_class = cosine_similarity(image_features.cpu().detach().numpy() , queries_matrix )

    threshold += maximum_similarity

100%|██████████| 1120/1120 [01:02<00:00, 18.04it/s]


In [28]:
threshold /= 1120
threshold

0.8226966007036057

In [29]:
test_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/test/images'

for file_name in tqdm(os.listdir(test_path)) :

    image = cv2.imread(f'{test_path}/{file_name}')
    image = refine_image(image)

    inputs = processor(images = image , return_tensors = 'pt').to(device)
    image_features = model.get_image_features(** inputs)

    maximum_similarity , target_class = cosine_similarity(image_features.cpu().detach().numpy() , queries_matrix )

    if maximum_similarity < threshold :

        target_class = 22

    submissions['img_file'].append(file_name)
    submissions['class'].append(target_class)

100%|██████████| 1120/1120 [00:46<00:00, 24.14it/s]


In [30]:
submissions = pd.DataFrame.from_dict(submissions)
submissions_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/submissions/submissions.csv'

with open(submissions_path, 'w') as csv_file:

    submissions.to_csv(path_or_buf = csv_file , index = False)

In [31]:
submissions.groupby('class').count()

,img_file
class,
0,4
1,114
2,1
3,2
4,24
5,4
6,63
7,68
8,2


In [32]:
!kaggle competitions submit -c image-search -f '/content/drive/MyDrive/SuperAI_SS4/SuperAI_SS4_Level_1/Hack_1_Image_Search/submissions/submission.csv' -m "submit"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/kaggle/cli.py", line 54, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 806, in competition_submit_cli
    submit_result = self.competition_submit(file_name, message,
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 757, in competition_submit
    content_length=os.path.getsize(file_name),
  File "/usr/lib/python3.10/genericpath.py", line 50, in getsize
    return os.stat(filename).st_size
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/SuperAI_SS4/SuperAI_SS4_Level_1/Hack_1_Image_Search/submissions/submissions.csv'
